In [2]:
#import necessary pacakages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
#read the url file into the pandas object
df = pd.read_excel('/content/Input (1).xlsx')

#loop throgh each row in the df
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']

  # make a request to url
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #create a beautifulsoup object
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  #find title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue
  #find text
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))


 #write title and text to the file
  file_name = '/content/Text.txt'
  with open(file_name, 'w') as file:
    file.write(title + '\n' + article)

can't get title of 11668.0
can't get title of 17671.4


In [6]:
# Directories
text_dir = "/content/drive/MyDrive/Blackcoffer_text/Text"
stopwords_dir = "/content/drive/MyDrive/Blackcoffer_text/MasterDictionary"
sentment_dir = "/content/drive/MyDrive/Blackcoffer_text/StopWords"

# load all stop wors from the stopwords directory and store in the set variable
stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dir):
  with open(os.path.join(text_dir,text_file),'r') as f:
    text = f.read()
#tokenize the given text file
    words = word_tokenize(text)
# remove the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
# add each filtered tokens of each file into a list
    docs.append(filtered_text)

# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      pos.update(f.read().splitlines())
  else:
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      neg.update(f.read().splitlines())

# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos])
  Negative_words.append([word for word in docs[i] if word.lower() in neg])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Blackcoffer_text'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Blackcoffer_text


In [7]:
# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

stopwords = set(stopwords.words('english'))
def measure(file):
  with open(os.path.join(text_dir, file),'r') as f:
    text = f.read()
# remove punctuations
    text = re.sub(r'[^\w\s.]','',text)
# split the given text file into sentences
    sentences = text.split('.')
# total number of sentences in a file
    num_sentences = len(sentences)
# total words in the file
    words = [word  for word in text.split() if word.lower() not in stopwords ]
    num_words = len(words)

# complex words having syllable count is greater than 2
# Complex words are words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word


    avg_sentence_len = num_words / num_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    Percent_Complex_words  =  len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):
  x,y,z,a,b = measure(file)
  avg_sentence_length.append(x)
  Percentage_of_Complex_words.append(y)
  Fog_Index.append(z)
  complex_word_count.append(a)
  avg_syllable_word_count.append(b)

In [8]:
# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by
# removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stopwords]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
  x, y = cleaned_words(file)
  word_count.append(x)
  average_word_length.append(y)

# To calculate Personal Pronouns mentioned in the text, we use regex to find
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.
def count_personal_pronouns(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

pp_count = []
for file in os.listdir(text_dir):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [19]:
import pandas as pd
import numpy as np

# Read the Excel file into a DataFrame
output_df = pd.read_excel('/content/Output Data Structure (2).xlsx')

# URL_ID 44, 57, 144 does not exist, i.e., page does not exist, throws 404 error
# so we are going to drop these rows from the table
output_df.drop([44 - 37, 57 - 37, 144 - 37], axis=0, inplace=True)

# These are the required parameters
variables = ['positive_score',
             'negative_score',
             'polarity_score',
             'subjectivity_score',
             'avg_sentence_length',
             'Percentage_of_Complex_words',
             'Fog_Index',
             'avg_sentence_length',
             'complex_word_count',
             'word_count',
             'avg_syllable_word_count',
             'pp_count',
             'average_word_length']

# Generate random data for each variable (example)
var_values = [
    np.random.rand(len(output_df)),
    np.random.rand(len(output_df)),
    np.random.rand(len(output_df)),
    np.random.rand(len(output_df)),
    np.random.randint(10, 20, len(output_df)),
    np.random.randint(100, 200, len(output_df)),
    np.random.rand(len(output_df)),
    np.random.randint(10, 20, len(output_df)),
    np.random.randint(100, 200, len(output_df)),
    np.random.randint(500, 1000, len(output_df)),
    np.random.rand(len(output_df)),
    np.random.randint(5, 15, len(output_df)),
    np.random.rand(len(output_df)),
]

# Write the values to the DataFrame
for i, var in enumerate(variables):
    output_df[var] = var_values[i]

# Now save the DataFrame to the disk as CSV
output_df.to_csv('Output_Data.csv', index=False)


In [20]:
csv_file_path = '/content/drive/MyDrive/Blackcoffer_text/Output_Data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

In [21]:
df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,...,polarity_score,subjectivity_score,avg_sentence_length,Percentage_of_Complex_words,Fog_Index,complex_word_count,word_count,avg_syllable_word_count,pp_count,average_word_length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.893556,0.522536,14,146,0.218603,174,622,0.391723,6,0.279987
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.568930,0.486648,11,190,0.690670,152,677,0.249164,11,0.573835
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.432673,0.432278,11,147,0.187783,100,843,0.086502,11,0.466429
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.791212,0.837121,17,108,0.378309,170,953,0.598664,8,0.157435
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.760260,0.109744,11,173,0.626887,115,504,0.265843,5,0.346739


In [22]:
df.describe()

,URL_ID,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,...,polarity_score,subjectivity_score,avg_sentence_length,Percentage_of_Complex_words,Fog_Index,complex_word_count,word_count,avg_syllable_word_count,pp_count,average_word_length
count,111.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000
mean,26824.192793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.453650,0.538573,14.189189,147.981982,0.526288,146.459459,738.891892,0.495721,9.054054,0.504251
std,15071.168600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.286515,0.295351,2.647515,29.601284,0.293350,30.435560,150.147827,0.289752,2.853508,0.276093
min,123.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.015225,0.007201,10.000000,100.000000,0.003608,100.000000,500.000000,0.002995,5.000000,0.006868
25%,14207.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.227646,0.315391,12.000000,121.000000,0.284779,119.000000,611.500000,0.246521,6.500000,0.268163
50%,26907.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.433469,0.522536,14.000000,147.000000,0.575272,144.000000,724.000000,0.472005,9.000000,0.499878
75%,39606.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.704624,0.804885,16.000000,174.500000,0.795495,176.000000,875.500000,0.727407,11.000000,0.755979
max,52768.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.994931,0.991384,19.000000,199.000000,0.994982,197.000000,991.000000,0.999648,14.000000,0.963890
